# ETL for Covid-19 data in Mexico

**Objective**: This notebook outlines the ETL (Extract, Transform, Load) process for managing COVID-19 data from Mexico. The primary goal is to extract the daily published raw data from Mexico's Ministry of Health (*Secretaria de Salud*), transform it into a clean and structured format suitable for analysis, and load it into a structured data format that can be utilized in subsequent analysis and modeling tasks.

Given the continuous changes and updates in the published data, it is essential to work with a reliable data source that preserves historical records. Therefore, we will rely on a digital archive that captures past versions of the data, ensuring consistency and reproducibility in our analysis. The data used in this ETL process can be accessed through the digital archive at the following link: [COVID-19 Data Archive](https://web.archive.org/web/20220122063317/http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/08/datos_abiertos_covid19_30.08.2020.zip).

**Process**:
1. **Extraction**:
   - Data is extracted from the archived versions of daily reports published by Mexico's Ministry of Health. This step involves accessing the archive, downloading raw data files, and ensuring data completeness and integrity.
   
2. **Transformation**:
   - The raw data undergoes a cleaning and transformation process, including handling missing values, correcting data types, standardizing date formats, and calculating derived metrics such as daily new cases and deaths.
   - This step also involves filtering relevant information, such as specific states or time periods, and enriching the data with additional contextual information.

3. **Loading**:
   - The transformed data is loaded into a structured format, such as CSV files or databases, to be used in exploratory data analysis (EDA), modeling, and visualization.
   - This organized data provides a solid foundation for understanding the progression of COVID-19 in Mexico and supports data-driven decision-making.

**Importance**: The ETL process ensures that the data used for analysis is reliable, up-to-date, and in a format conducive to accurate analysis. By leveraging archived data, we can maintain consistency in our analysis despite ongoing updates and changes to the original published data, thereby enhancing the validity of our results.

In [1]:
# Import all necesary libraries
import os
import pandas as pd
import numpy as np
from datetime import datetime
from itertools import product

## Delay in COVID-19 Reporting

In this first section, we will perform the Extract and Transform (ET) steps for the COVID-19 data from August 1st, 2020. Having successfully understood the data, we will repeat the same steps for the published databases of August 7 and August 14 of 2020, corresponding to data published 1 week and 2 weeks later.

#### August 1, 2020 data

In [120]:
# Read the data
raw_data_2020_08_01 = pd.read_csv('/Users/ro/Downloads/200801COVID19MEXICO.csv', encoding='latin1', low_memory=False)

# Convert object columns to datetime format
raw_data_2020_08_01['FECHA_INGRESO'] = pd.to_datetime(raw_data_2020_08_01['FECHA_INGRESO'], errors='coerce') 
raw_data_2020_08_01['FECHA_SINTOMAS'] = pd.to_datetime(raw_data_2020_08_01['FECHA_SINTOMAS'], errors='coerce') 

# Replace '9999-99-99' with NaT (Not a Time)
raw_data_2020_08_01['FECHA_DEF'] = raw_data_2020_08_01['FECHA_DEF'].replace('9999-99-99', pd.NaT)

# Convert FECHA_DEF to datetime after replacement
raw_data_2020_08_01['FECHA_DEF'] = pd.to_datetime(raw_data_2020_08_01['FECHA_DEF'], errors='coerce')

In [121]:
raw_data_2020_08_01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999697 entries, 0 to 999696
Data columns (total 35 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   FECHA_ACTUALIZACION  999697 non-null  object        
 1   ID_REGISTRO          999697 non-null  object        
 2   ORIGEN               999697 non-null  int64         
 3   SECTOR               999697 non-null  int64         
 4   ENTIDAD_UM           999697 non-null  int64         
 5   SEXO                 999697 non-null  int64         
 6   ENTIDAD_NAC          999697 non-null  int64         
 7   ENTIDAD_RES          999697 non-null  int64         
 8   MUNICIPIO_RES        999697 non-null  int64         
 9   TIPO_PACIENTE        999697 non-null  int64         
 10  FECHA_INGRESO        999697 non-null  datetime64[ns]
 11  FECHA_SINTOMAS       999697 non-null  datetime64[ns]
 12  FECHA_DEF            61264 non-null   datetime64[ns]
 13  INTUBADO      

In [122]:
raw_data_2020_08_01.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-08-01,0e23f9,2,3,15,1,15,15,99,1,...,2,2,2,2,2,1,99,MÃ©xico,99,97
1,2020-08-01,14c60f,2,3,15,2,15,15,106,2,...,2,2,2,2,2,1,99,MÃ©xico,99,2
2,2020-08-01,1b640f,2,4,9,2,9,9,11,2,...,2,1,2,2,99,1,99,MÃ©xico,99,2
3,2020-08-01,0c8a89,2,4,15,2,15,15,109,2,...,2,2,2,2,99,1,99,MÃ©xico,99,2
4,2020-08-01,159028,2,4,7,1,9,7,97,1,...,2,2,2,2,99,1,99,MÃ©xico,99,97


In [123]:
raw_data_2020_08_01.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [124]:
# Filter the data to include confirmed and unknown cases (excluding those with RESULTADO = 2, which represent negative cases)
confirmed_cases_2020_08_01 = raw_data_2020_08_01[raw_data_2020_08_01['RESULTADO'] != 2].copy()

# Calculate the number of days from symptom onset to death
# FECHA_DEF: Date of death
# FECHA_SINTOMAS: Date of symptom onset
confirmed_cases_2020_08_01['OnsetToDeath'] = (confirmed_cases_2020_08_01['FECHA_DEF'] - confirmed_cases_2020_08_01['FECHA_SINTOMAS']).dt.days

# Calculate the number of days from symptom onset to hospital admission
# FECHA_INGRESO: Date of hospital admission
confirmed_cases_2020_08_01['OnsetToHospital'] = (confirmed_cases_2020_08_01['FECHA_INGRESO'] - confirmed_cases_2020_08_01['FECHA_SINTOMAS']).dt.days

# Filter out cases where OnsetToDeath is negative, keeping only realistic values (positive or zero days)
confirmed_cases_2020_08_01 = confirmed_cases_2020_08_01[confirmed_cases_2020_08_01['OnsetToDeath'] >= 0].copy()

# Filter out cases where OnsetToHospital is negative, keeping only realistic values (positive or zero days)
confirmed_cases_2020_08_01 = confirmed_cases_2020_08_01[confirmed_cases_2020_08_01['OnsetToHospital'] >= 0].copy()


confirmed_cases_2020_08_01.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,OnsetToDeath,OnsetToHospital
3,2020-08-01,0c8a89,2,4,15,2,15,15,109,2,...,2,2,99,1,99,MÃ©xico,99,2,10.0,8
13,2020-08-01,175782,2,4,2,2,30,2,3,2,...,2,1,99,1,99,MÃ©xico,99,2,6.0,0
15,2020-08-01,15179a,2,4,8,1,8,8,37,2,...,1,2,99,1,99,MÃ©xico,99,2,4.0,4
16,2020-08-01,1745d4,2,4,15,2,9,15,33,2,...,2,2,99,1,99,MÃ©xico,99,2,7.0,0
20,2020-08-01,071720,2,4,9,2,9,9,10,2,...,2,2,99,1,99,MÃ©xico,99,2,21.0,0


In [125]:
# Group confirmed cases by the date of symptom onset (FECHA_SINTOMAS) and the state of residence (ENTIDAD_RES)
state_cases_2020_08_01 = confirmed_cases_2020_08_01.groupby(['FECHA_SINTOMAS', 'ENTIDAD_RES']).size().reset_index(name='confirmed_cases')

# Group confirmed deaths by the date of death (FECHA_DEF) and the state of residence (ENTIDAD_RES)
state_deaths_2020_08_01 = confirmed_cases_2020_08_01.groupby(['FECHA_DEF', 'ENTIDAD_RES']).size().reset_index(name='confirmed_deaths')

# Rename the date columns in both DataFrames to a common name 'FECHA'
# This is done to facilitate merging the cases and deaths on the same date column
state_cases_2020_08_01.rename(columns={'FECHA_SINTOMAS': 'FECHA'}, inplace=True)
state_deaths_2020_08_01.rename(columns={'FECHA_DEF': 'FECHA'}, inplace=True)

# Merge the confirmed cases and confirmed deaths DataFrames on the date ('FECHA') and state ('ENTIDAD_RES')
processed_data_2020_08_01= pd.merge(state_cases_2020_08_01, state_deaths_2020_08_01, how='outer', on=['FECHA', 'ENTIDAD_RES']).fillna(0)

# Rename the merged columns for clarity and consistency: 'FECHA' to 'date' and 'ENTIDAD_RES' to 'state'
processed_data_2020_08_01.rename(columns={'FECHA': 'date', 'ENTIDAD_RES': 'state'}, inplace=True)

processed_data_2020_08_01.head()


,date,state,confirmed_cases,confirmed_deaths
0,2020-02-26,15,1.0,0.0
1,2020-02-27,9,1.0,0.0
2,2020-03-01,25,1.0,0.0
3,2020-03-02,15,1.0,0.0
4,2020-03-03,9,1.0,0.0


In [126]:
# Create a list of all unique dates in the data
all_dates_2020_08_01 = pd.date_range(start=processed_data_2020_08_01['date'].min(), 
                          end=processed_data_2020_08_01['date'].max(), 
                          freq='D')

# Create a sorted list of all unique states in your data
all_states_2020_08_01 = sorted(processed_data_2020_08_01['state'].unique())

print(all_dates_2020_08_01)
print(all_states_2020_08_01)

DatetimeIndex(['2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29',
               '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04',
               '2020-03-05', '2020-03-06',
               ...
               '2020-07-23', '2020-07-24', '2020-07-25', '2020-07-26',
               '2020-07-27', '2020-07-28', '2020-07-29', '2020-07-30',
               '2020-07-31', '2020-08-01'],
              dtype='datetime64[ns]', length=158, freq='D')
[np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32)]


In [133]:
# Create a complete template DataFrame with all possible combinations of dates and states
complete_template_2020_08_01 = pd.DataFrame(list(product(all_dates_2020_08_01, all_states_2020_08_01)), columns=['date', 'state'])

# Merge the complete template with the state_time_series DataFrame
final_data_2020_08_01 = pd.merge(complete_template_2020_08_01, processed_data_2020_08_01, on=['date', 'state'], how='left')

# Fill missing values in the 'confirmed_cases' column with 0
final_data_2020_08_01['confirmed_cases'] = final_data_2020_08_01['confirmed_cases'].fillna(0)

# Fill missing values in the 'confirmed_deaths' column with 0
final_data_2020_08_01['confirmed_deaths'] = final_data_2020_08_01['confirmed_deaths'].fillna(0)

final_data_2020_08_01.tail(32)

,date,state,confirmed_cases,confirmed_deaths
5024,2020-08-01,1,0.0,0.0
5025,2020-08-01,2,0.0,0.0
5026,2020-08-01,3,0.0,0.0
5027,2020-08-01,4,0.0,0.0
5028,2020-08-01,5,0.0,0.0
5029,2020-08-01,6,0.0,0.0
5030,2020-08-01,7,0.0,2.0
5031,2020-08-01,8,0.0,0.0
5032,2020-08-01,9,0.0,0.0
5033,2020-08-01,10,0.0,0.0


In [165]:
# Save the DataFrame as a pickle file in the Data folder for later use
data_folder = '/Users/ro/Desktop/Undergrad_AM_Thesis/Data'

file_path = os.path.join(data_folder, 'final_data_2020_08_01.pkl')
final_data_2020_08_01.to_pickle(file_path)

#### August 8, 2020 data

In [134]:
# Read the data
raw_data_2020_08_08 = pd.read_csv('/Users/ro/Downloads/200808COVID19MEXICO.csv', encoding='latin1', low_memory=False)

# Convert object columns to datetime format
raw_data_2020_08_08['FECHA_INGRESO'] = pd.to_datetime(raw_data_2020_08_08['FECHA_INGRESO'], errors='coerce') 
raw_data_2020_08_08['FECHA_SINTOMAS'] = pd.to_datetime(raw_data_2020_08_08['FECHA_SINTOMAS'], errors='coerce') 

# Replace '9999-99-99' with NaT (Not a Time)
raw_data_2020_08_08['FECHA_DEF'] = raw_data_2020_08_08['FECHA_DEF'].replace('9999-99-99', pd.NaT)

# Convert FECHA_DEF to datetime after replacement
raw_data_2020_08_08['FECHA_DEF'] = pd.to_datetime(raw_data_2020_08_08['FECHA_DEF'], errors='coerce')

raw_data_2020_08_08.tail()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
1085892,2020-08-08,16f9f8,1,12,28,2,28,28,38,1,...,2,2,2,2,2,3,99,MÃ©xico,99,97
1085893,2020-08-08,156a86,2,12,9,2,9,9,11,2,...,98,98,98,98,1,3,99,MÃ©xico,99,2
1085894,2020-08-08,02a9f9,2,4,19,1,19,19,39,1,...,2,1,2,2,99,3,99,MÃ©xico,99,97
1085895,2020-08-08,110e67,2,12,9,1,9,9,7,1,...,2,2,2,2,2,3,99,MÃ©xico,99,97
1085896,2020-08-08,02668d,2,12,15,2,15,15,109,1,...,2,2,2,2,2,3,99,MÃ©xico,99,97


In [135]:
raw_data_2020_08_08.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1085897 entries, 0 to 1085896
Data columns (total 35 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   FECHA_ACTUALIZACION  1085897 non-null  object        
 1   ID_REGISTRO          1085897 non-null  object        
 2   ORIGEN               1085897 non-null  int64         
 3   SECTOR               1085897 non-null  int64         
 4   ENTIDAD_UM           1085897 non-null  int64         
 5   SEXO                 1085897 non-null  int64         
 6   ENTIDAD_NAC          1085897 non-null  int64         
 7   ENTIDAD_RES          1085897 non-null  int64         
 8   MUNICIPIO_RES        1085897 non-null  int64         
 9   TIPO_PACIENTE        1085897 non-null  int64         
 10  FECHA_INGRESO        1085897 non-null  datetime64[ns]
 11  FECHA_SINTOMAS       1085897 non-null  datetime64[ns]
 12  FECHA_DEF            66818 non-null    datetime64[ns]
 1

In [136]:
# Filter the data to include confirmed and unknown cases (excluding those with RESULTADO = 2, which represent negative cases)
confirmed_cases_2020_08_08 = raw_data_2020_08_08[raw_data_2020_08_08['RESULTADO'] != 2].copy()

# Calculate the number of days from symptom onset to death
# FECHA_DEF: Date of death
# FECHA_SINTOMAS: Date of symptom onset
confirmed_cases_2020_08_08['OnsetToDeath'] = (confirmed_cases_2020_08_08['FECHA_DEF'] - confirmed_cases_2020_08_08['FECHA_SINTOMAS']).dt.days

# Calculate the number of days from symptom onset to hospital admission
# FECHA_INGRESO: Date of hospital admission
confirmed_cases_2020_08_08['OnsetToHospital'] = (confirmed_cases_2020_08_08['FECHA_INGRESO'] - confirmed_cases_2020_08_08['FECHA_SINTOMAS']).dt.days

# Filter out cases where OnsetToDeath is negative, keeping only realistic values (positive or zero days)
confirmed_cases_2020_08_08 = confirmed_cases_2020_08_08[confirmed_cases_2020_08_08['OnsetToDeath'] >= 0].copy()

# Filter out cases where OnsetToHospital is negative, keeping only realistic values (positive or zero days)
confirmed_cases_2020_08_08 = confirmed_cases_2020_08_08[confirmed_cases_2020_08_08['OnsetToHospital'] >= 0].copy()


confirmed_cases_2020_08_08.tail()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,OnsetToDeath,OnsetToHospital
1085568,2020-08-08,1aeae8,2,12,29,2,29,29,25,2,...,2,2,99,3,99,MÃ©xico,99,1,3.0,3
1085654,2020-08-08,0feb99,1,12,9,2,9,15,58,2,...,1,1,1,3,99,MÃ©xico,99,1,36.0,2
1085870,2020-08-08,067f58,1,6,29,2,29,29,33,2,...,2,2,2,3,99,MÃ©xico,99,2,11.0,7
1085893,2020-08-08,156a86,2,12,9,2,9,9,11,2,...,98,98,1,3,99,MÃ©xico,99,2,1.0,0
1085894,2020-08-08,02a9f9,2,4,19,1,19,19,39,1,...,2,2,99,3,99,MÃ©xico,99,97,7.0,5


In [137]:
# Group confirmed cases by the date of symptom onset (FECHA_SINTOMAS) and the state of residence (ENTIDAD_RES)
state_cases_2020_08_08 = confirmed_cases_2020_08_08.groupby(['FECHA_SINTOMAS', 'ENTIDAD_RES']).size().reset_index(name='confirmed_cases')

# Group confirmed deaths by the date of death (FECHA_DEF) and the state of residence (ENTIDAD_RES)
state_deaths_2020_08_08 = confirmed_cases_2020_08_08.groupby(['FECHA_DEF', 'ENTIDAD_RES']).size().reset_index(name='confirmed_deaths')

# Rename the date columns in both DataFrames to a common name 'FECHA'
# This is done to facilitate merging the cases and deaths on the same date column
state_cases_2020_08_08.rename(columns={'FECHA_SINTOMAS': 'FECHA'}, inplace=True)
state_deaths_2020_08_08.rename(columns={'FECHA_DEF': 'FECHA'}, inplace=True)

# Merge the confirmed cases and confirmed deaths DataFrames on the date ('FECHA') and state ('ENTIDAD_RES')
processed_data_2020_08_08= pd.merge(state_cases_2020_08_08, state_deaths_2020_08_08, how='outer', on=['FECHA', 'ENTIDAD_RES']).fillna(0)

# Rename the merged columns for clarity and consistency: 'FECHA' to 'date' and 'ENTIDAD_RES' to 'state'
processed_data_2020_08_08.rename(columns={'FECHA': 'date', 'ENTIDAD_RES': 'state'}, inplace=True)

processed_data_2020_08_08.tail()

,date,state,confirmed_cases,confirmed_deaths
4192,2020-08-07,30,0.0,7.0
4193,2020-08-07,31,0.0,6.0
4194,2020-08-07,32,0.0,1.0
4195,2020-08-08,20,0.0,1.0
4196,2020-08-08,27,0.0,1.0


In [160]:
# Create a list of all unique dates in the data
all_dates_2020_08_08 = pd.date_range(start=processed_data_2020_08_08['date'].min(), 
                          end=processed_data_2020_08_08['date'].max(), 
                          freq='D')

# Create a sorted list of all unique states in your data
all_states_2020_08_08 = sorted(processed_data_2020_08_08['state'].unique())

print(all_dates_2020_08_08)
print(all_states_2020_08_08)

DatetimeIndex(['2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29',
               '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04',
               '2020-03-05', '2020-03-06',
               ...
               '2020-07-30', '2020-07-31', '2020-08-01', '2020-08-02',
               '2020-08-03', '2020-08-04', '2020-08-05', '2020-08-06',
               '2020-08-07', '2020-08-08'],
              dtype='datetime64[ns]', length=165, freq='D')
[np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32)]


In [161]:
# Create a complete template DataFrame with all possible combinations of dates and states
complete_template_2020_08_08 = pd.DataFrame(list(product(all_dates_2020_08_08, all_states_2020_08_08)), columns=['date', 'state'])

# Merge the complete template with the state_time_series DataFrame
final_data_2020_08_08 = pd.merge(complete_template_2020_08_08, processed_data_2020_08_08, on=['date', 'state'], how='left')

# Fill missing values in the 'confirmed_cases' column with 0
final_data_2020_08_08['confirmed_cases'] = final_data_2020_08_08['confirmed_cases'].fillna(0)

# Fill missing values in the 'confirmed_deaths' column with 0
final_data_2020_08_08['confirmed_deaths'] = final_data_2020_08_08['confirmed_deaths'].fillna(0)

final_data_2020_08_08[final_data_2020_08_08['date'] == datetime(2020,8,1)]

,date,state,confirmed_cases,confirmed_deaths
5024,2020-08-01,1,0.0,2.0
5025,2020-08-01,2,2.0,12.0
5026,2020-08-01,3,1.0,3.0
5027,2020-08-01,4,1.0,11.0
5028,2020-08-01,5,6.0,29.0
5029,2020-08-01,6,1.0,9.0
5030,2020-08-01,7,1.0,7.0
5031,2020-08-01,8,2.0,6.0
5032,2020-08-01,9,9.0,38.0
5033,2020-08-01,10,1.0,4.0


In [166]:
# Save the DataFrame as a pickle file in the Data folder for later use
data_folder = '/Users/ro/Desktop/Undergrad_AM_Thesis/Data'

file_path = os.path.join(data_folder, 'final_data_2020_08_08.pkl')
final_data_2020_08_08.to_pickle(file_path)

#### August 15, 2020 data

In [154]:
# Read the data
raw_data_2020_08_15 = pd.read_csv('/Users/ro/Downloads/200815COVID19MEXICO.csv', encoding='latin1', low_memory=False)

# Convert object columns to datetime format
raw_data_2020_08_15['FECHA_INGRESO'] = pd.to_datetime(raw_data_2020_08_15['FECHA_INGRESO'], errors='coerce') 
raw_data_2020_08_15['FECHA_SINTOMAS'] = pd.to_datetime(raw_data_2020_08_15['FECHA_SINTOMAS'], errors='coerce') 

# Replace '9999-99-99' with NaT (Not a Time)
raw_data_2020_08_15['FECHA_DEF'] = raw_data_2020_08_15['FECHA_DEF'].replace('9999-99-99', pd.NaT)

# Convert FECHA_DEF to datetime after replacement
raw_data_2020_08_15['FECHA_DEF'] = pd.to_datetime(raw_data_2020_08_15['FECHA_DEF'], errors='coerce')

raw_data_2020_08_15.tail()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
1171002,2020-08-15,1d7eaa,1,12,22,1,22,22,14,1,...,2,2,2,2,1,3,99,MÃ©xico,99,97
1171003,2020-08-15,0cc9a3,2,4,10,1,10,10,23,1,...,2,2,2,2,99,3,99,MÃ©xico,99,97
1171004,2020-08-15,0f436c,2,9,19,2,19,19,48,2,...,2,2,2,2,2,3,99,MÃ©xico,99,2
1171005,2020-08-15,146669,1,12,23,2,20,23,5,2,...,2,2,2,2,2,3,99,MÃ©xico,99,1
1171006,2020-08-15,1a44af,2,12,8,1,8,8,32,1,...,2,2,2,2,1,3,99,MÃ©xico,99,97


In [155]:
raw_data_2020_08_15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1171007 entries, 0 to 1171006
Data columns (total 35 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   FECHA_ACTUALIZACION  1171007 non-null  object        
 1   ID_REGISTRO          1171007 non-null  object        
 2   ORIGEN               1171007 non-null  int64         
 3   SECTOR               1171007 non-null  int64         
 4   ENTIDAD_UM           1171007 non-null  int64         
 5   SEXO                 1171007 non-null  int64         
 6   ENTIDAD_NAC          1171007 non-null  int64         
 7   ENTIDAD_RES          1171007 non-null  int64         
 8   MUNICIPIO_RES        1171007 non-null  int64         
 9   TIPO_PACIENTE        1171007 non-null  int64         
 10  FECHA_INGRESO        1171007 non-null  datetime64[ns]
 11  FECHA_SINTOMAS       1171007 non-null  datetime64[ns]
 12  FECHA_DEF            72324 non-null    datetime64[ns]
 1

In [156]:
# Filter the data to include confirmed and unknown cases (excluding those with RESULTADO = 2, which represent negative cases)
confirmed_cases_2020_08_15 = raw_data_2020_08_15[raw_data_2020_08_15['RESULTADO'] != 2].copy()

# Calculate the number of days from symptom onset to death
# FECHA_DEF: Date of death
# FECHA_SINTOMAS: Date of symptom onset
confirmed_cases_2020_08_15['OnsetToDeath'] = (confirmed_cases_2020_08_15['FECHA_DEF'] - confirmed_cases_2020_08_15['FECHA_SINTOMAS']).dt.days

# Calculate the number of days from symptom onset to hospital admission
# FECHA_INGRESO: Date of hospital admission
confirmed_cases_2020_08_15['OnsetToHospital'] = (confirmed_cases_2020_08_15['FECHA_INGRESO'] - confirmed_cases_2020_08_15['FECHA_SINTOMAS']).dt.days

# Filter out cases where OnsetToDeath is negative, keeping only realistic values (positive or zero days)
confirmed_cases_2020_08_15 = confirmed_cases_2020_08_15[confirmed_cases_2020_08_15['OnsetToDeath'] >= 0].copy()

# Filter out cases where OnsetToHospital is negative, keeping only realistic values (positive or zero days)
confirmed_cases_2020_08_15 = confirmed_cases_2020_08_15[confirmed_cases_2020_08_15['OnsetToHospital'] >= 0].copy()


confirmed_cases_2020_08_15.tail()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,OnsetToDeath,OnsetToHospital
1170429,2020-08-15,0016ab,1,4,9,1,9,9,4,2,...,2,2,99,3,99,MÃ©xico,99,2,7.0,4
1170489,2020-08-15,1519c8,1,3,15,1,15,15,60,2,...,2,2,1,3,99,MÃ©xico,99,2,1.0,1
1170655,2020-08-15,1d295f,2,4,11,2,11,11,20,2,...,2,2,99,3,99,MÃ©xico,99,2,8.0,7
1170671,2020-08-15,16c120,2,4,15,2,15,15,104,2,...,2,2,99,3,99,MÃ©xico,99,2,8.0,0
1170746,2020-08-15,19b2f2,2,4,14,2,14,14,108,2,...,2,2,99,3,99,MÃ©xico,99,2,13.0,5


In [157]:
# Group confirmed cases by the date of symptom onset (FECHA_SINTOMAS) and the state of residence (ENTIDAD_RES)
state_cases_2020_08_15 = confirmed_cases_2020_08_15.groupby(['FECHA_SINTOMAS', 'ENTIDAD_RES']).size().reset_index(name='confirmed_cases')

# Group confirmed deaths by the date of death (FECHA_DEF) and the state of residence (ENTIDAD_RES)
state_deaths_2020_08_15 = confirmed_cases_2020_08_15.groupby(['FECHA_DEF', 'ENTIDAD_RES']).size().reset_index(name='confirmed_deaths')

# Rename the date columns in both DataFrames to a common name 'FECHA'
# This is done to facilitate merging the cases and deaths on the same date column
state_cases_2020_08_15.rename(columns={'FECHA_SINTOMAS': 'FECHA'}, inplace=True)
state_deaths_2020_08_15.rename(columns={'FECHA_DEF': 'FECHA'}, inplace=True)

# Merge the confirmed cases and confirmed deaths DataFrames on the date ('FECHA') and state ('ENTIDAD_RES')
processed_data_2020_08_15 = pd.merge(state_cases_2020_08_15, state_deaths_2020_08_15, how='outer', on=['FECHA', 'ENTIDAD_RES']).fillna(0)

# Rename the merged columns for clarity and consistency: 'FECHA' to 'date' and 'ENTIDAD_RES' to 'state'
processed_data_2020_08_15.rename(columns={'FECHA': 'date', 'ENTIDAD_RES': 'state'}, inplace=True)

processed_data_2020_08_15.tail()

,date,state,confirmed_cases,confirmed_deaths
4416,2020-08-14,29,0.0,1.0
4417,2020-08-14,30,0.0,8.0
4418,2020-08-14,31,0.0,5.0
4419,2020-08-14,32,0.0,1.0
4420,2020-08-15,5,0.0,1.0


In [158]:
# Create a list of all unique dates in the data
all_dates_2020_08_15 = pd.date_range(start=processed_data_2020_08_15['date'].min(), 
                          end=processed_data_2020_08_15['date'].max(), 
                          freq='D')

# Create a sorted list of all unique states in your data
all_states_2020_08_15 = sorted(processed_data_2020_08_15['state'].unique())

print(all_dates_2020_08_15)
print(all_states_2020_08_15)

DatetimeIndex(['2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29',
               '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04',
               '2020-03-05', '2020-03-06',
               ...
               '2020-08-06', '2020-08-07', '2020-08-08', '2020-08-09',
               '2020-08-10', '2020-08-11', '2020-08-12', '2020-08-13',
               '2020-08-14', '2020-08-15'],
              dtype='datetime64[ns]', length=172, freq='D')
[np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32)]


In [162]:
# Create a complete template DataFrame with all possible combinations of dates and states
complete_template_2020_08_15 = pd.DataFrame(list(product(all_dates_2020_08_15, all_states_2020_08_15)), columns=['date', 'state'])

# Merge the complete template with the state_time_series DataFrame
final_data_2020_08_15 = pd.merge(complete_template_2020_08_15, processed_data_2020_08_15, on=['date', 'state'], how='left')

# Fill missing values in the 'confirmed_cases' column with 0
final_data_2020_08_15['confirmed_cases'] = final_data_2020_08_15['confirmed_cases'].fillna(0)

# Fill missing values in the 'confirmed_deaths' column with 0
final_data_2020_08_15['confirmed_deaths'] = final_data_2020_08_15['confirmed_deaths'].fillna(0)

final_data_2020_08_15[final_data_2020_08_15['date'] == datetime(2020,8,1)]

,date,state,confirmed_cases,confirmed_deaths
5024,2020-08-01,1,2.0,3.0
5025,2020-08-01,2,4.0,13.0
5026,2020-08-01,3,3.0,5.0
5027,2020-08-01,4,4.0,11.0
5028,2020-08-01,5,23.0,30.0
5029,2020-08-01,6,3.0,9.0
5030,2020-08-01,7,2.0,7.0
5031,2020-08-01,8,8.0,7.0
5032,2020-08-01,9,26.0,39.0
5033,2020-08-01,10,5.0,4.0


In [167]:
# Save as a pickle file in the Data folder for later use
data_folder = '/Users/ro/Desktop/Undergrad_AM_Thesis/Data'

file_path = os.path.join(data_folder, 'final_data_2020_08_15.pkl')
final_data_2020_08_15.to_pickle(file_path)

## Consolidated Covid-19 dattabase for EDA and modelling

In this first section, we will perform the Extract and Transform (ET) steps for the COVID-19 data up to the end of 2023. All the used databases can be found in Mexico's *Secretaria de Salud* [Open Data Historic Databases ](https://www.gob.mx/salud/documentos/datos-abiertos-bases-historicas-direccion-general-de-epidemiologia).

Due to constant changes in available data and to avoid issues related to delays in reporting, we will use the following databases for each years data:
* 2020 data corresponds to the public database published in 2021-10-31
* 2021 data corresponds to the public database published in 2022-08-09
* 2022 data corresponds to the public database published in 2023-04-25
* 2023 data corresponds to the public database published in 2024-04-30

In [60]:
DB_names = ['COVID19MEXICO-7.csv','230425COVID19MEXICO.csv','COVID19MEXICO2021.csv','COVID19MEXICO2020.csv']

# Read the data
raw_data_2020 = pd.read_csv('/Users/ro/Downloads/COVID19MEXICO2020.csv', encoding='latin1', low_memory=False)
raw_data_2021 = pd.read_csv('/Users/ro/Downloads/COVID19MEXICO2021.csv', encoding='latin1', low_memory=False)
raw_data_2022 = pd.read_csv('/Users/ro/Downloads/230425COVID19MEXICO.csv', encoding='latin1', low_memory=False)
raw_data_2023 = pd.read_csv('/Users/ro/Downloads/COVID19MEXICO-7.csv', encoding='latin1', low_memory=False)

In [74]:
# Convert object columns to datetime format
raw_data_2020['FECHA_INGRESO'] = pd.to_datetime(raw_data_2020['FECHA_INGRESO'], errors='coerce')
raw_data_2020['FECHA_SINTOMAS'] = pd.to_datetime(raw_data_2020['FECHA_SINTOMAS'], errors='coerce')

raw_data_2021['FECHA_INGRESO'] = pd.to_datetime(raw_data_2021['FECHA_INGRESO'], errors='coerce')
raw_data_2021['FECHA_SINTOMAS'] = pd.to_datetime(raw_data_2021['FECHA_SINTOMAS'], errors='coerce')

raw_data_2022['FECHA_INGRESO'] = pd.to_datetime(raw_data_2022['FECHA_INGRESO'], errors='coerce')
raw_data_2022['FECHA_SINTOMAS'] = pd.to_datetime(raw_data_2022['FECHA_SINTOMAS'], errors='coerce')

raw_data_2023['FECHA_INGRESO'] = pd.to_datetime(raw_data_2023['FECHA_INGRESO'], errors='coerce')
raw_data_2023['FECHA_SINTOMAS'] = pd.to_datetime(raw_data_2023['FECHA_SINTOMAS'], errors='coerce')

# Replace '9999-99-99' with NaT (Not a Time)
raw_data_2020['FECHA_DEF'] = raw_data_2020['FECHA_DEF'].replace('9999-99-99', pd.NaT)
raw_data_2020['FECHA_DEF'] = pd.to_datetime(raw_data_2020['FECHA_DEF'], errors='coerce')

raw_data_2021['FECHA_DEF'] = raw_data_2021['FECHA_DEF'].replace('9999-99-99', pd.NaT)
raw_data_2021['FECHA_DEF'] = pd.to_datetime(raw_data_2021['FECHA_DEF'], errors='coerce')

raw_data_2022['FECHA_DEF'] = raw_data_2022['FECHA_DEF'].replace('9999-99-99', pd.NaT)
raw_data_2022['FECHA_DEF'] = pd.to_datetime(raw_data_2022['FECHA_DEF'], errors='coerce')

raw_data_2023['FECHA_DEF'] = raw_data_2023['FECHA_DEF'].replace('9999-99-99', pd.NaT)
raw_data_2023['FECHA_DEF'] = pd.to_datetime(raw_data_2023['FECHA_DEF'], errors='coerce')

In [6]:
# Filter only confirmed Covid-19 cases
confirmed_cases_2020 = raw_data_2020[raw_data_2020['CLASIFICACION_FINAL'].isin([1, 2, 3])].copy()
confirmed_cases_2021 = raw_data_2021[raw_data_2021['CLASIFICACION_FINAL'].isin([1, 2, 3])].copy()
confirmed_cases_2022 = raw_data_2022[raw_data_2022['CLASIFICACION_FINAL'].isin([1, 2, 3])].copy()
confirmed_cases_2023 = raw_data_2023[raw_data_2023['CLASIFICACION_FINAL'].isin([1, 2, 3])].copy()

In [154]:
# Calculate time intervals from symptom onset to death and to hospital admission for confirmed cases across 2020-2023, filtering out negative values to retain only logical timeframes (we follow the logic of https://datos.covid-19.conacyt.mx/)
confirmed_cases_2020['OnsetToDeath'] = (confirmed_cases_2020['FECHA_DEF'] - confirmed_cases_2020['FECHA_SINTOMAS']).dt.days
confirmed_cases_2020['OnsetToHospital'] = (confirmed_cases_2020['FECHA_INGRESO'] - confirmed_cases_2020['FECHA_SINTOMAS']).dt.days
confirmed_cases_2020 = confirmed_cases_2020[confirmed_cases_2020['OnsetToDeath'] >= 0].copy()
confirmed_cases_2020 = confirmed_cases_2020[confirmed_cases_2020['OnsetToHospital'] >= 0].copy()

confirmed_cases_2021['OnsetToDeath'] = (confirmed_cases_2021['FECHA_DEF'] - confirmed_cases_2021['FECHA_SINTOMAS']).dt.days
confirmed_cases_2021['OnsetToHospital'] = (confirmed_cases_2021['FECHA_INGRESO'] - confirmed_cases_2021['FECHA_SINTOMAS']).dt.days
confirmed_cases_2021 = confirmed_cases_2021[confirmed_cases_2021['OnsetToDeath'] >= 0].copy()
confirmed_cases_2021 = confirmed_cases_2021[confirmed_cases_2021['OnsetToHospital'] >= 0].copy()

confirmed_cases_2022['OnsetToDeath'] = (confirmed_cases_2022['FECHA_DEF'] - confirmed_cases_2022['FECHA_SINTOMAS']).dt.days
confirmed_cases_2022['OnsetToHospital'] = (confirmed_cases_2022['FECHA_INGRESO'] - confirmed_cases_2022['FECHA_SINTOMAS']).dt.days
confirmed_cases_2022 = confirmed_cases_2022[confirmed_cases_2022['OnsetToDeath'] >= 0].copy()
confirmed_cases_2022 = confirmed_cases_2022[confirmed_cases_2022['OnsetToHospital'] >= 0].copy()

confirmed_cases_2023['OnsetToDeath'] = (confirmed_cases_2023['FECHA_DEF'] - confirmed_cases_2023['FECHA_SINTOMAS']).dt.days
confirmed_cases_2023['OnsetToHospital'] = (confirmed_cases_2023['FECHA_INGRESO'] - confirmed_cases_2023['FECHA_SINTOMAS']).dt.days
confirmed_cases_2023 = confirmed_cases_2023[confirmed_cases_2023['OnsetToDeath'] >= 0].copy()
confirmed_cases_2023 = confirmed_cases_2023[confirmed_cases_2023['OnsetToHospital'] >= 0].copy()

In [226]:
# Consolidate Onset to Death data in a new Dataframe for further exploration 
onset_to_death_2020 = confirmed_cases_2020[['FECHA_SINTOMAS', 'FECHA_DEF', 'OnsetToDeath']].copy()
onset_to_death_2020['year'] = 2020

onset_to_death_2021 = confirmed_cases_2021[['FECHA_SINTOMAS', 'FECHA_DEF', 'OnsetToDeath']].copy()
onset_to_death_2021['year'] = 2021

onset_to_death_2022 = confirmed_cases_2022[['FECHA_SINTOMAS', 'FECHA_DEF', 'OnsetToDeath']].copy()
onset_to_death_2022['year'] = 2022

onset_to_death_2023 = confirmed_cases_2023[['FECHA_SINTOMAS', 'FECHA_DEF', 'OnsetToDeath']].copy()
onset_to_death_2023['year'] = 2023

# Concatenate all the data into a single DataFrame
onset_to_death_df = pd.concat([onset_to_death_2020, onset_to_death_2021, onset_to_death_2022, onset_to_death_2023], ignore_index=True)

# Save as a pickle file in the Data folder for later use
data_folder = '/Users/ro/Desktop/Undergrad_AM_Thesis/Data'

file_path = os.path.join(data_folder, 'onset_to_death_df.pkl')
onset_to_death_df.to_pickle(file_path)

In [227]:
# Consolidate Onset to Hospital data in a new DataFrame for further exploration
onset_to_hospital_2020 = confirmed_cases_2020[['FECHA_SINTOMAS', 'FECHA_INGRESO', 'OnsetToHospital']].copy()
onset_to_hospital_2020['year'] = 2020

onset_to_hospital_2021 = confirmed_cases_2021[['FECHA_SINTOMAS', 'FECHA_INGRESO', 'OnsetToHospital']].copy()
onset_to_hospital_2021['year'] = 2021

onset_to_hospital_2022 = confirmed_cases_2022[['FECHA_SINTOMAS', 'FECHA_INGRESO', 'OnsetToHospital']].copy()
onset_to_hospital_2022['year'] = 2022

onset_to_hospital_2023 = confirmed_cases_2023[['FECHA_SINTOMAS', 'FECHA_INGRESO', 'OnsetToHospital']].copy()
onset_to_hospital_2023['year'] = 2023

# Concatenate all the data into a single DataFrame
onset_to_hospital_df = pd.concat([onset_to_hospital_2020, onset_to_hospital_2021, onset_to_hospital_2022, onset_to_hospital_2023], ignore_index=True)

# Save as a pickle file in the Data folder for later use
data_folder = '/Users/ro/Desktop/Undergrad_AM_Thesis/Data'
file_path = os.path.join(data_folder, 'onset_to_hospital_df.pkl')
onset_to_hospital_df.to_pickle(file_path)

In [134]:
# Group confirmed cases by the date of symptom onset (FECHA_SINTOMAS) and the state of residence (ENTIDAD_RES)
state_cases_2020 = confirmed_cases_2020.groupby(['FECHA_SINTOMAS', 'ENTIDAD_RES']).size().reset_index(name='confirmed_cases')
state_cases_2021 = confirmed_cases_2021.groupby(['FECHA_SINTOMAS', 'ENTIDAD_RES']).size().reset_index(name='confirmed_cases')
state_cases_2022 = confirmed_cases_2022.groupby(['FECHA_SINTOMAS', 'ENTIDAD_RES']).size().reset_index(name='confirmed_cases')
state_cases_2023 = confirmed_cases_2023.groupby(['FECHA_SINTOMAS', 'ENTIDAD_RES']).size().reset_index(name='confirmed_cases')


# Group confirmed deaths by the date of death (FECHA_DEF) and the state of residence (ENTIDAD_RES)
state_deaths_2020 = confirmed_cases_2020.groupby(['FECHA_DEF', 'ENTIDAD_RES']).size().reset_index(name='confirmed_deaths')
state_deaths_2021 = confirmed_cases_2021.groupby(['FECHA_DEF', 'ENTIDAD_RES']).size().reset_index(name='confirmed_deaths')
state_deaths_2022 = confirmed_cases_2022.groupby(['FECHA_DEF', 'ENTIDAD_RES']).size().reset_index(name='confirmed_deaths')
state_deaths_2023 = confirmed_cases_2023.groupby(['FECHA_DEF', 'ENTIDAD_RES']).size().reset_index(name='confirmed_deaths')


# Rename the date columns in both DataFrames to a common name 'FECHA'
state_cases_2020.rename(columns={'FECHA_SINTOMAS': 'date'}, inplace=True)
state_deaths_2020.rename(columns={'FECHA_DEF': 'date'}, inplace=True)

state_cases_2021.rename(columns={'FECHA_SINTOMAS': 'date'}, inplace=True)
state_deaths_2021.rename(columns={'FECHA_DEF': 'date'}, inplace=True)

state_cases_2022.rename(columns={'FECHA_SINTOMAS': 'date'}, inplace=True)
state_deaths_2022.rename(columns={'FECHA_DEF': 'date'}, inplace=True)

state_cases_2023.rename(columns={'FECHA_SINTOMAS': 'date'}, inplace=True)
state_deaths_2023.rename(columns={'FECHA_DEF': 'date'}, inplace=True)

# Merge the confirmed cases and confirmed deaths DataFrames on the date ('FECHA') and state ('ENTIDAD_RES')
processed_data_2020 = pd.merge(state_cases_2020, state_deaths_2020, how='outer', on=['date', 'ENTIDAD_RES']).fillna(0)
processed_data_2021 = pd.merge(state_cases_2021, state_deaths_2021, how='outer', on=['date', 'ENTIDAD_RES']).fillna(0)
processed_data_2022 = pd.merge(state_cases_2022, state_deaths_2022, how='outer', on=['date', 'ENTIDAD_RES']).fillna(0)
processed_data_2023 = pd.merge(state_cases_2023, state_deaths_2023, how='outer', on=['date', 'ENTIDAD_RES']).fillna(0)

# Rename columns for simplicity
processed_data_2020.rename(columns={'ENTIDAD_RES': 'state'}, inplace=True)
processed_data_2021.rename(columns={'ENTIDAD_RES': 'state'}, inplace=True)
processed_data_2022.rename(columns={'ENTIDAD_RES': 'state'}, inplace=True)
processed_data_2023.rename(columns={'ENTIDAD_RES': 'state'}, inplace=True)

In [176]:
# We will get the time series at national scale in order to understand what is happening with the data at the end/begining of the year
national_2020 = processed_data_2020.groupby('date').agg({
    'confirmed_cases': 'sum',
    'confirmed_deaths': 'sum'
}).reset_index()

national_2021 = processed_data_2021.groupby('date').agg({
    'confirmed_cases': 'sum',
    'confirmed_deaths': 'sum'
}).reset_index()

national_2022 = processed_data_2022.groupby('date').agg({
    'confirmed_cases': 'sum',
    'confirmed_deaths': 'sum'
}).reset_index()

national_2023 = processed_data_2023.groupby('date').agg({
    'confirmed_cases': 'sum',
    'confirmed_deaths': 'sum'
}).reset_index()

In [177]:
# Only death counts are carried into the next year, leaving all confirmed cases at 0
max_data_2020 = national_2020['date'].max()
national_2020[national_2020['date']>=pd.Timestamp(2020,12,28)][0:10]

,date,confirmed_cases,confirmed_deaths
301,2020-12-28,1363.0,964.0
302,2020-12-29,964.0,971.0
303,2020-12-30,1090.0,946.0
304,2020-12-31,1302.0,886.0
305,2021-01-01,0.0,974.0
306,2021-01-02,0.0,963.0
307,2021-01-03,0.0,981.0
308,2021-01-04,0.0,1019.0
309,2021-01-05,0.0,923.0
310,2021-01-06,0.0,932.0


In [178]:
# Cconfirmed deaths appear to be too low, signaling that our previous hypothesis is correct
national_2021[national_2021['date'] <= max_data_2020][0:10]

,date,confirmed_cases,confirmed_deaths
0,2021-01-01,2202.0,13.0
1,2021-01-02,1248.0,30.0
2,2021-01-03,1370.0,56.0
3,2021-01-04,1496.0,93.0
4,2021-01-05,1547.0,146.0
5,2021-01-06,1394.0,228.0
6,2021-01-07,1358.0,274.0
7,2021-01-08,1436.0,365.0
8,2021-01-09,1225.0,431.0
9,2021-01-10,1857.0,519.0


When comparing case counts across databases at the end/begining of the next year it appears that confirmed cases and deaths from previous years are not carried on to the following year. To solve for this inconsistency we shall add both case counts in our merge.

In [180]:
processed_data_2020

,date,state,confirmed_cases,confirmed_deaths
0,2020-02-26,15,1.0,0.0
1,2020-02-29,15,1.0,0.0
2,2020-03-01,25,1.0,0.0
3,2020-03-02,15,2.0,0.0
4,2020-03-03,9,1.0,0.0
...,...,...,...,...
10047,2021-09-11,17,0.0,1.0
10048,2021-09-14,30,0.0,1.0
10049,2021-09-15,15,0.0,1.0
10050,2021-09-25,30,0.0,1.0


In [204]:
# ------------------------------------------------
# Merge consecutive years: 2020 and 2021
merged_2020_2021 = pd.merge(processed_data_2020, processed_data_2021, on=['date', 'state'], how='outer', suffixes=('_2020', '_2021')).fillna(0)

# Sum confirmed cases and deaths for overlapping dates
merged_2020_2021['confirmed_cases'] = merged_2020_2021['confirmed_cases_2020'] + merged_2020_2021['confirmed_cases_2021']
merged_2020_2021['confirmed_deaths'] = merged_2020_2021['confirmed_deaths_2020'] + merged_2020_2021['confirmed_deaths_2021']

# Drop the temporary columns after summing
merged_2020_2021 = merged_2020_2021.drop(['confirmed_cases_2020', 'confirmed_cases_2021', 'confirmed_deaths_2020', 'confirmed_deaths_2021'], axis=1)


# ------------------------------------------------
# Merge consecutive years: 2022 and 2023
merged_2022_2023 = pd.merge(processed_data_2022, processed_data_2023, on=['date', 'state'], how='outer', suffixes=('_2022', '_2023')).fillna(0)

# Sum confirmed cases and deaths for overlapping dates
merged_2022_2023['confirmed_cases'] = merged_2022_2023['confirmed_cases_2022'] + merged_2022_2023['confirmed_cases_2023']
merged_2022_2023['confirmed_deaths'] = merged_2022_2023['confirmed_deaths_2022'] + merged_2022_2023['confirmed_deaths_2023']

# Drop the temporary columns after summing
merged_2022_2023 = merged_2022_2023.drop(['confirmed_cases_2022', 'confirmed_cases_2023', 'confirmed_deaths_2022', 'confirmed_deaths_2023'], axis=1)


# ------------------------------------------------
# Merge consecutive years: 2020-2021 with 2022-2023
processed_data_final = pd.merge(merged_2020_2021, merged_2022_2023, on=['date', 'state'], how='outer', suffixes=('_2021', '_2022')).fillna(0)

# Sum confirmed cases and deaths for overlapping dates
processed_data_final['confirmed_cases'] = processed_data_final['confirmed_cases_2021'] + processed_data_final['confirmed_cases_2022']
processed_data_final['confirmed_deaths'] = processed_data_final['confirmed_cases_2021'] + processed_data_final['confirmed_cases_2022']

# Drop the temporary columns after summing
processed_data_final = processed_data_final.drop(['confirmed_cases_2021', 'confirmed_cases_2022', 'confirmed_deaths_2021', 'confirmed_deaths_2022'], axis=1)

# Display the final merged DataFrame
processed_data_final

,date,state,confirmed_cases,confirmed_deaths
0,2020-02-26,15,1.0,1.0
1,2020-02-29,15,1.0,1.0
2,2020-03-01,25,1.0,1.0
3,2020-03-02,15,2.0,2.0
4,2020-03-03,9,1.0,1.0
...,...,...,...,...
30620,2024-04-24,15,1.0,1.0
30621,2024-04-25,4,0.0,0.0
30622,2024-04-25,15,0.0,0.0
30623,2024-04-27,13,0.0,0.0


In [179]:
# Create a complete template DataFrame with all possible combinations of dates and states
all_dates = pd.date_range(start=confirmed_cases_2020['FECHA_SINTOMAS'].min(), 
                          end=pd.Timestamp(2023,12,31), 
                          freq='D')

all_states = list(range(1, 33))

complete_template = pd.DataFrame(list(product(all_dates, all_states)), columns=['date', 'state'])

complete_template

,date,state
0,2020-02-26,1
1,2020-02-26,2
2,2020-02-26,3
3,2020-02-26,4
4,2020-02-26,5
...,...,...
44955,2023-12-31,28
44956,2023-12-31,29
44957,2023-12-31,30
44958,2023-12-31,31


In [228]:
# Merge the complete template with the procesed_data_final DataFrame
covid_data_df = pd.merge(complete_template, processed_data_final, on=['date', 'state'], how='left').fillna(0)
covid_data_df

,date,state,confirmed_cases,confirmed_deaths
0,2020-02-26,1,0.0,0.0
1,2020-02-26,2,0.0,0.0
2,2020-02-26,3,0.0,0.0
3,2020-02-26,4,0.0,0.0
4,2020-02-26,5,0.0,0.0
...,...,...,...,...
44955,2023-12-31,28,0.0,0.0
44956,2023-12-31,29,0.0,0.0
44957,2023-12-31,30,0.0,0.0
44958,2023-12-31,31,0.0,0.0


In [229]:
# Save as a pickle file in the Data folder for later use
data_folder = '/Users/ro/Desktop/Undergrad_AM_Thesis/Data'
file_path = os.path.join(data_folder, 'covid_data_df.pkl')
covid_data_df.to_pickle(file_path)